In [1]:
import pandas as pd
import ipywidgets as ipw
from scipy.stats import gmean
import numpy as np
import networkx as nx

predictions = pd.read_pickle("data/global_ind_pheno_tumor_munge.best_pred.00.pickle")
for chunk in range(1, 9):
    predictions = pd.concat([predictions, pd.read_pickle("data/global_ind_pheno_tumor_munge.best_pred.%02d.pickle" % chunk)])
hp_annot = pd.read_pickle("data/global_ind_pheno_tumor_munge.best.pickle")
omim = pd.read_pickle("data/OMIM_annotation_data.pickle")
hpo_net = nx.gpickle.read_gpickle("data/hp.180127.obo.gpickle")

gene_list = ipw.Textarea(
    placeholder='Enter your gene list',
    description='Genes:',
    disabled=False
)


search_widget = ipw.Text()
hpo_id = list(hpo_net.nodes())
hpo_id.sort()
hpo_terms = [hpo_net.node[x]['name'] for x in hpo_id]
full_terms = ["%s: %s" % (hpo_id[x], hpo_terms[x]) for x in range(len(hpo_id))]
options_widget = ipw.SelectMultiple(options=full_terms)
add_button = ipw.Button(description='Add term')
remove_button = ipw.Button(description='Remove term')
selected_widget = ipw.SelectMultiple()
left_area = ipw.VBox([options_widget, add_button])
right_area = ipw.VBox([selected_widget, remove_button])
selection_area = ipw.HBox([left_area, right_area])
multi_select = ipw.VBox([search_widget, selection_area])
selected_terms = set()
        # Wire the search field to the checkboxes
def on_text_change(change):
    search_input = change['new']
    if search_input == '':
        # Reset search field
        new_options = full_terms
    else:
        # Filter by search field using difflib.
        new_options = [x for x in full_terms if search_input.lower() in x.lower()]
    options_widget.options = new_options

def on_add(b):
    new_options = set([x for x in selected_widget.options] + [x for x in options_widget.value])
    new_options = list(new_options)
    new_options.sort()
    selected_widget.options = new_options

add_button.on_click(on_add)    

def on_remove(b):
    v = [x for x in selected_widget.options if not x in selected_widget.value]
    selected_widget.options = v

remove_button.on_click(on_remove)    

search_widget.observe(on_text_change, names='value')

display(gene_list, multi_select)


#    return multi_select